# Practical Exercise 2.03: Descriptive analysis of market indexes

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define symbols
symbols = ["^GSPC", "AGG"]  # S&P 500 and US Aggregate Bond ETF

# Define date range
start_date = "2010-01-01"
end_date = None  # Gets the most recent data available

# Fetch data from Yahoo Finance

data = yf.download(symbols, start=start_date, end=end_date, auto_adjust=False, actions=False)['Adj Close']


# Rename columns for clarity
data.columns = ["US Aggregate Bond ETF", "S&P 500"]


# Compute moving averages
data["SMA_50"] = data["S&P 500"].rolling(50).mean()
data["SMA_200"] = data["S&P 500"].rolling(200).mean()
data["EMA_50"] = data["S&P 500"].ewm(span=50, adjust=False).mean()

# Compute daily returns
returns = data.pct_change().dropna()

# Compute risk and performance metrics
def annualized_metrics(returns):
    annualized_return = returns.mean() * 252
    annualized_volatility = returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_volatility
    return annualized_return, annualized_volatility, sharpe_ratio

sp500_metrics = annualized_metrics(returns["S&P 500"])
bond_metrics = annualized_metrics(returns["US Aggregate Bond ETF"])

print("\n Performance Metrics:")
print(f"S&P 500 - Annualized Return: {sp500_metrics[0]:.2%}, Volatility: {sp500_metrics[1]:.2%}, Sharpe Ratio: {sp500_metrics[2]:.2f}")
print(f"US Aggregate Bond ETF - Annualized Return: {bond_metrics[0]:.2%}, Volatility: {bond_metrics[1]:.2%}, Sharpe Ratio: {bond_metrics[2]:.2f}")

# Compute drawdown analysis
def drawdown_analysis(series):
    cumulative = (1 + series).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    return drawdown

data["Drawdown S&P 500"] = drawdown_analysis(returns["S&P 500"])
data["Drawdown Bonds"] = drawdown_analysis(returns["US Aggregate Bond ETF"])

# Compute rolling volatility
data["Rolling Volatility"] = returns["S&P 500"].rolling(50).std() * np.sqrt(252)

# Plot S&P 500 with moving averages
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.set_xlabel("Date")
ax1.set_ylabel("S&P 500 Price", color="grey")
ax1.plot(data.index, data["S&P 500"], label="S&P 500", color="black")
ax1.plot(data.index, data["SMA_50"], label="50-Day SMA", linestyle="--", color="blue")
ax1.plot(data.index, data["SMA_200"], label="200-Day SMA", linestyle="--", color="red")
ax1.plot(data.index, data["EMA_50"], label="50-Day EMA", linestyle="--", color="green")
ax1.tick_params(axis="y", labelcolor="black")

# Create secondary y-axis for Bonds
ax2 = ax1.twinx()
ax2.set_ylabel("US Aggregate Bond ETF Price", color="blue")
ax2.plot(data.index, data["US Aggregate Bond ETF"], label="US Aggregate Bond ETF", color="blue")
ax2.tick_params(axis="y", labelcolor="blue")

# Title, legend, and grid
plt.title("S&P 500 vs US Aggregate Bond ETF - Adj Closing Prices & Trend Indicators")
fig.tight_layout()
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
plt.grid()
plt.show()

# Plot rolling volatility
plt.figure(figsize=(12, 6))
plt.plot(data.index, data["Rolling Volatility"], label="50-Day Rolling Volatility", color="purple")
plt.title("S&P 500 Rolling Volatility (50-Day Window)")
plt.legend()
plt.grid(True)
plt.show()

# Plot drawdowns
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(data.index, data["Drawdown S&P 500"], label="S&P 500 Drawdown", color="black")
ax.plot(data.index, data["Drawdown Bonds"], label="US Aggregate Bond ETF Drawdown", color="blue")
ax.set_ylabel("Drawdown (%)")
ax.set_xlabel("Date")
ax.set_title("Drawdown Analysis: S&P 500 vs US Aggregate Bond ETF")
ax.axhline(0, color="red", linestyle="--", linewidth=1)
ax.legend()
plt.grid(True)
plt.show()

# Compute rolling correlation
data["Rolling Correlation"] = returns["S&P 500"].rolling(50).corr(returns["US Aggregate Bond ETF"])

# Plot rolling correlation
plt.figure(figsize=(12, 6))
plt.plot(data.index, data["Rolling Correlation"], label="Rolling Correlation (S&P 500 vs. Bonds)", color="orange")
plt.axhline(0, color="black", linestyle="--", linewidth=1)
plt.title("Rolling Correlation between S&P 500 and US Aggregate Bond ETF Returns (50-Day Window)")
plt.legend()
plt.grid(True)
plt.show()
